<a href="https://colab.research.google.com/github/shreyash53/SMAI-Knowledge-Distilation/blob/main/KD_MNIST_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
# Importing MNIST dataset

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("Train data shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

Train data shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
from sklearn.preprocessing import OneHotEncoder

print ('label shape: ',np.shape(y_train),np.shape(y_test))
def encode(y):
    df = pd.DataFrame(y, columns=["class"])
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(df[['class']]).toarray())
    final_df = df.join(encoder_df)
    final_df.drop('class', axis=1, inplace=True)
    return final_df.to_numpy()
y_train = encode(y_train)
y_test = encode(y_test)
print ('label shape: ',np.shape(y_train),np.shape(y_test))
# print(y_test_[2], y_test[2])

label shape:  (60000,) (10000,)
label shape:  (60000, 10) (10000, 10)


In [ ]:
# Teacher Model

teacher_model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(1200, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1200, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
    ], name="teacher"
)

teacher_model.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 1200)              942000    
                                                                 
 dropout (Dropout)           (None, 1200)              0         
                                                                 
 dense_1 (Dense)             (None, 1200)              1441200   
                                                                 
 dropout_1 (Dropout)         (None, 1200)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                12010     
                                                                 
Total params: 2,395,210
Trainable params: 2,395,210
Non-tra

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
teacher_model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
teacher_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[callback])

In [ ]:
y_test.shape

(10000, 10)

In [ ]:
# Evaluate Teacher Model

score = teacher_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.07354190945625305
Test accuracy: 0.9768000245094299


In [3]:
# Small Model

small_model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(800, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(800, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"), 
    ], name="Student"
)

small_model.summary()

Model: "Student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 800)               628000    
                                                                 
 dropout (Dropout)           (None, 800)               0         
                                                                 
 dense_1 (Dense)             (None, 800)               640800    
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                8010      
                                                                 
Total params: 1,276,810
Trainable params: 1,276,810
Non-tra

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
small_model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
small_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[callback])

Epoch 1/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.7825 - accuracy: 0.7619 - val_loss: 0.2705 - val_accuracy: 0.9222
Epoch 2/20
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3925 - accuracy: 0.8835 - val_loss: 0.2045 - val_accuracy: 0.9370
Epoch 3/20
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3171 - accuracy: 0.9057 - val_loss: 0.1717 - val_accuracy: 0.9505
Epoch 4/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2689 - accuracy: 0.9202 - val_loss: 0.1493 - val_accuracy: 0.9593
Epoch 5/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2369 - accuracy: 0.9293 - val_loss: 0.1328 - val_accuracy: 0.9643
Epoch 6/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2154 - accuracy: 0.9352 - val_loss: 0.1206 - val_accuracy: 0.9668
Epoch 7/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1911 - accuracy: 0.9427 - val_loss: 0.1105 - val_accuracy:

In [ ]:
# Evaluate Small Model

score = small_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.07492998987436295
Test accuracy: 0.9764999747276306


In [ ]:
# Student Model
class StudentModel(keras.Model):
    def __init__(self, T):
        super(StudentModel, self).__init__()
        
        self.T = T
        
        self.input_layer = keras.Input(shape=(28, 28, 1))
        self.flat = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(800, activation="relu")
        self.do1 = keras.layers.Dropout(0.5)
        self.d2 = keras.layers.Dense(800, activation="relu")
        self.do2 = keras.layers.Dropout(0.5)
        self.d3 = keras.layers.Dense(10, activation="softmax")

    def call(self, x):
        x = self.flat(x)
        x = self.d1(x)
        x = self.do1(x)
        x = self.d2(x)
        x = self.do2(x)
        x = self.d3(x / self.T)
        return x

In [ ]:
temp = 8
student_model = StudentModel(temp)

loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

student_model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=["accuracy"])

In [ ]:
y_pred_teacher = teacher_model.predict(x_train)

student_model.fit(x_train, y_pred_teacher, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3162 - accuracy: 0.9080
Epoch 2/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1485 - accuracy: 0.9630
Epoch 3/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1314 - accuracy: 0.9708
Epoch 4/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1222 - accuracy: 0.9748
Epoch 5/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1189 - accuracy: 0.9764
Epoch 6/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1165 - accuracy: 0.9776
Epoch 7/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1121 - accuracy: 0.9808
Epoch 8/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1125 - accuracy: 0.9799
Epoch 9/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1091 - accuracy: 0.9810
Epoch 10/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.108

In [ ]:
# Evaluate Student model
score = student_model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.07544821500778198
Test accuracy: 0.9769999980926514


In [ ]:
x_test_no_3 = []
y_test_no_3 = []
x_test_3 = []
y_test_3 = []

for i in range(len(x_test)):
    if np.argmax(y_test[i]) == 3:
        x_test_3.append(x_test[i])
        y_test_3.append(y_test[i])
        # print(y_test[i])
    else:
        x_test_no_3.append(x_test[i])
        y_test_no_3.append(y_test[i])

x_test_no_3 = np.array(x_test_no_3)
y_test_no_3 = np.array(y_test_no_3)
x_test_3 = np.array(x_test_3)
y_test_3 = np.array(y_test_3)

In [ ]:
x_train_no_3 = []
y_train_no_3 = []
x_train_3 = []
y_train_3 = []

for i in range(len(x_train)):
    if np.argmax(y_train[i]) == 3:
        x_train_3.append(x_train[i])
        y_train_3.append(y_train[i])
    else:
        x_train_no_3.append(x_train[i])
        y_train_no_3.append(y_train[i])

x_train_no_3 = np.array(x_train_no_3)
y_train_no_3 = np.array(y_train_no_3)
x_train_3 = np.array(x_train_3)
y_train_3 = np.array(y_train_3)

In [ ]:
len(x_test_3)

1010

In [ ]:
temp = 6
student_model = StudentModel(temp)

loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

student_model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=["accuracy"])

In [ ]:
y_pred_teacher_no_3 = teacher_model.predict(x_train_no_3)

student_model.fit(x_train_no_3, y_pred_teacher_no_3, epochs=20)

Epoch 1/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.2348 - accuracy: 0.9347
Epoch 2/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1255 - accuracy: 0.9725
Epoch 3/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1140 - accuracy: 0.9778
Epoch 4/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1084 - accuracy: 0.9806
Epoch 5/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1045 - accuracy: 0.9818
Epoch 6/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1026 - accuracy: 0.9824
Epoch 7/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.1016 - accuracy: 0.9844
Epoch 8/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.0995 - accuracy: 0.9847
Epoch 9/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.0999 - accuracy: 0.9836
Epoch 10/20
1684/1684 [==============================] - 8s 5ms/step - loss: 0.097

In [ ]:
# Evaluate Student model
score = student_model.evaluate(x_test_3, y_test_3, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.8868426084518433
Test accuracy: 0.5970296859741211
